In [6]:
#4,5 
import numpy as np

# تعریف پارامترها
L = 0.7  # طول سیستم
s_in = 0.01
s_out = 2 * L

pi = np.pi
num_steps = 10000

# محاسبات اولیه
k = (2 * pi) / L
K = k**2 / 8

# محاسبه A_H و A_L
A_L = 0.00750
A_H = 0.0080
N = 1

# مقادیر g و g_p و g_m
g = K * ((L / pi) ** 2)
g_p = np.sqrt(1 + g)
g_m = np.sqrt(1 - g)

# تابع مثلثاتی مشترک
def trig_functions(s, g_p, g_m, L):
    bar_s = (np.pi * s) / L
    Cp = np.cos(g_p * bar_s)
    Cm = np.cos(g_m * bar_s)
    Sp = np.sin(g_p * bar_s)
    Sm = np.sin(g_m * bar_s)
    return Cp, Cm, Sp, Sm

# محاسبه ماتریس M_core
def compute_M_core(L, g, g_p, g_m, s):
    Cp, Cm, Sp, Sm = trig_functions(s, g_p, g_m, L)
    M_core = np.array([
        [Cp, (L / (g * np.pi)) * ((g_p * Sp) - (Sm / g_m)), Sm / g_m, (L / (g * np.pi)) * (Cm - Cp)],
        [(-np.pi * g * Sp) / (L * g_p), Cp, 0, (Sp / g_p)],
        [(-Sp / g_p), (L / (g * np.pi)) * (Cp - Cm), Cm, (L / (g * np.pi)) * ((Sp / g_p) - (g_m * Sm))],
        [0, -(Sm / g_m), (np.pi * g * Sm) / (L * g_m), Cm]
    ])
    return M_core

# محاسبه ماتریس‌های چرخش R_positive و R_negative
def compute_R_positive(k, s):
    return np.array([
        [np.cos(k * s_out / 2), 0, -np.sin(k * s_out / 2), 0],
        [0, np.cos(k * s_out / 2), 0, -np.sin(k * s_out / 2)],
        [np.sin(k * s_out / 2), 0, np.cos(k * s_out / 2), 0],
        [0, np.sin(k * s_out / 2), 0, np.cos(k * s_out / 2)]
    ])

def compute_R_negative(k, s):
    return np.array([
        [np.cos(k * s_in / 2 * N), 0, np.sin(k * s_in / 2 * N), 0],
        [0, np.cos(k * s_in / 2 * N), 0, np.sin(k * s_in / 2 * N)],
        [-np.sin(k * s_in / 2 * N), 0, np.cos(k * s_in / 2 * N), 0],
        [0, -np.sin(k * s_in / 2 * N), 0, np.cos(k * s_in / 2 * N)]
    ])

# محاسبه ماتریس انتقال M_HQ
def helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m):
    R_out = compute_R_positive(k, s_out)
    R_in = compute_R_negative(k, s_in)
    M_core = compute_M_core(L, g, g_p, g_m, s_out - s_in)
    M_HQ = R_out @ M_core @ R_in 
    return M_HQ

# تابع محاسبه‌ی ماتریس سیگما داخلی
def sigma_inner_matrix(k, K, A_H, A_L):
    sigma = np.zeros((4, 4))
    sigma[0, 0] = ((4 * k**2 + 4 * K) /(k**2)) * A_H**2 + (k**2 / K**2) * A_L**2
    sigma[1, 1] = 16 * (K**2 / k**2) * A_H**2
    sigma[2, 2] = 4 * A_H**2 + ((k**2 - 4 * K) / K**2) * A_L**2
    sigma[3, 3] = 4 * A_L**2
    sigma[0, 3] = sigma[3, 0] = 2 * (k / K) * A_L**2
    sigma[1, 2] = sigma[2, 1] = 8 * (K / k) * A_H**2
    return sigma

# اعمال ماتریس سیگما و محاسبه نهایی
def sigma_matrix(M_HQ, k, K, A_H, A_L):
    sigma_inner = sigma_inner_matrix(k, K, A_H, A_L)
    sigma_final = M_HQ @ sigma_inner @ M_HQ.T 
    return sigma_final 

# محاسبه انتشار
def compute_emittance(cov_matrix):
    return np.sqrt(np.linalg.det(cov_matrix))

# محاسبه پارامترهای توییس
def compute_twist_parameters(cov_matrix, emittance):
    beta = cov_matrix[0, 0] / emittance
    alpha = -cov_matrix[0, 1] / emittance
    gamma = cov_matrix[1, 1] / emittance
    return beta, alpha, gamma

# محاسبه M_HQ
M_HQ = helical_quadrupole_matrix(L, k, s_in, s_out, g, g_p, g_m)

# محاسبه ماتریس سیگما
sigma_final = sigma_matrix(M_HQ, k, K, A_H, A_L)

# جدا کردن ماتریس‌های کواریانس x و y
cov_x_final = sigma_final[:2, :2]
cov_y_final = sigma_final[2:, 2:]

# محاسبه پارامترهای نهایی برای x
emittance_x_final = compute_emittance(cov_x_final)
beta_x_final, alpha_x_final, gamma_x_final = compute_twist_parameters(cov_x_final, emittance_x_final)

# محاسبه پارامترهای نهایی برای y
emittance_y_final = compute_emittance(cov_y_final)
beta_y_final, alpha_y_final, gamma_y_final = compute_twist_parameters(cov_y_final, emittance_y_final)

# نمایش نتایج نهایی
print("\nFinal Twiss Parameters for x-plane (Transfer Matrix):")
print(f"Emittance: {emittance_x_final:.4f}")
print(f"Beta: {beta_x_final:.4f}")
print(f"Alpha: {alpha_x_final:.4f}")
print(f"Gamma: {gamma_x_final:.4f}")

print("\nFinal Twiss Parameters for y-plane (Transfer Matrix):")
print(f"Emittance: {emittance_y_final:.4f}")
print(f"Beta: {beta_y_final:.4f}")
print(f"Alpha: {alpha_y_final:.4f}")
print(f"Gamma: {gamma_y_final:.4f}")

# تابع محاسبه‌ی امیتنس بر اساس داده‌های ذرات (x و x')
def compute_emittance_from_data(positions, angles):
    mean_position = np.mean(positions)
    mean_angle = np.mean(angles)
    covariance = np.mean(positions * angles) - mean_position * mean_angle
    emittance = np.sqrt(np.mean(positions**2) * np.mean(angles**2) - covariance**2)
    return emittance

# تابع محاسبه‌ی پارامترهای توییس از داده‌ها
def compute_twist_parameters_from_data(positions, angles, emittance):
    if emittance == 0:
        return float('inf'), float('inf'), float('inf')  # Avoid division by zero
    beta = np.mean(positions**2) / emittance
    alpha = -np.mean(positions * angles) / emittance
    gamma = np.mean(angles**2) / emittance
    return beta, alpha, gamma

# تابع تغییر در track_particles
def track_particles(num_particles, k, s, K, A_H, A_L, x_init, x_prime_init, y_init, y_prime_init):
    deviations = np.random.multivariate_normal([0, 0, 0, 0], sigma_inner_matrix(k, K, A_H, A_L), num_particles).T
    
    x = x_init + deviations[0]
    x_prime = x_prime_init + deviations[1]
    y = y_init + deviations[2]
    y_prime = y_prime_init + deviations[3]
    
    return x, x_prime, y, y_prime

# ترکینگ ذرات
num_particles = 10000
x_init = 0.0153
x_prime_init = 0.01
y_init = 0.0146
y_prime_init = 0.01

x, x_prime, y, y_prime = track_particles(num_particles, k, s_out - s_in, K, A_H, A_L, x_init, x_prime_init, y_init, y_prime_init)

# محاسبه امیتنس و پارامترهای توییس
emittance_x_sim = compute_emittance_from_data(x, x_prime)
beta_x_sim, alpha_x_sim, gamma_x_sim = compute_twist_parameters_from_data(x, x_prime, emittance_x_sim)

emittance_y_sim = compute_emittance_from_data(y, y_prime)
beta_y_sim, alpha_y_sim, gamma_y_sim = compute_twist_parameters_from_data(y, y_prime, emittance_y_sim)

# نمایش نتایج ترکینگ
print("\ntracking Twiss Parameters for x-plane:")
print(f"Emittance: {emittance_x_sim:.4f}")
print(f"Beta: {beta_x_sim:.4f}")
print(f"Alpha: {alpha_x_sim:.4f}")
print(f"Gamma: {gamma_x_sim:.4f}")

print("\ntracking Twiss Parameters for y-plane:")
print(f"Emittance: {emittance_y_sim:.4f}")
print(f"Beta: {beta_y_sim:.4f}")
print(f"Alpha: {alpha_y_sim:.4f}")
print(f"Gamma: {gamma_y_sim:.4f}")



Final Twiss Parameters for x-plane (Transfer Matrix):
Emittance: 0.0007
Beta: 0.6892
Alpha: 0.0602
Gamma: 1.4562

Final Twiss Parameters for y-plane (Transfer Matrix):
Emittance: 0.0002
Beta: 0.9759
Alpha: 0.0946
Gamma: 1.0338

tracking Twiss Parameters for x-plane:
Emittance: 0.0009
Beta: 0.6354
Alpha: -0.1708
Gamma: 1.5737

tracking Twiss Parameters for y-plane:
Emittance: 0.0004
Beta: 1.2292
Alpha: -0.3715
Gamma: 0.8136
